In [1]:
# install required dependencies

%pip install --upgrade pip
%pip install boto3
%pip install botocore
%pip install numpy
%pip install pandas
%pip install python-dateutil
%pip install python-dotenv
%pip install urllib3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

In [1]:
# load the environment variables from .env file (in the same location as this .ipynb file)
%reload_ext dotenv
%dotenv
import os
import sys
os.getcwd()
print(os.getcwd() + '/scripts')
sys.path.append(os.getcwd() + '/scripts')
sys.path.append(os.getcwd() + '/scripts/external')

/data/scripts


In [2]:
# get the list of GDS path and sample_id
# TODO: make this operate over a list of sample names

# print(os.getenv('ICA_V1_TOKEN'))
# print(os.getenv('AWS_ACCESS_KEY_ID'))
# print(os.getenv('AWS_SECRET_ACCESS_KEY'))
# print(os.getenv('AWS_SESSION_TOKEN'))

from scripts.external import aws_session
from scripts.external import dynamo

session = aws_session.get_aws_session()

# get sample name
samples = ['NPM1000B52', 'NPM1000B53', 'NPM1000B54']
with open('metric_files/sample_list.txt', 'w') as f:
    f.write('\n'.join(samples))

# get GDS path
gds_paths_and_sample_ids = []
for sample in samples:
    sample_data = dynamo.query(pk=sample, sk_prefix='Data Transfer||Preparing||started||', index_name='gsi1pk-gsi1sk-index')
    additional_info = sample_data[0]['additionalinfo']
    gds_path = additional_info['gds_path']
    sample_id = additional_info['sample_id']
    print(gds_path)
    print(f'{sample}_{sample_id}')
    gds_paths_and_sample_ids.append((gds_path, f'{sample}_{sample_id}'))

metrics_folder = 'dragen-germline/output/'


gds://bssh.69fbb80cfe7635369366247c9c8b1a82/datasets/app.aps1-sh-prod.491493/
NPM1000B52_90119
gds://bssh.69fbb80cfe7635369366247c9c8b1a82/datasets/app.aps1-sh-prod.491495/
NPM1000B53_90120
gds://bssh.69fbb80cfe7635369366247c9c8b1a82/datasets/app.aps1-sh-prod.490525/
NPM1000B54_90121


In [3]:
# run metrics creation script over each sample's metrics CSVs
from glob import glob
import json
from scripts.external import icav1
from scripts import make_contractual_and_all_metrics as macam


for pair in gds_paths_and_sample_ids:
    files = icav1.list_files(gds_folder_path=f'{pair[0]}{metrics_folder}', token=os.getenv('ICA_V1_TOKEN'), page_size=100)
    # print(files)
    last_file_path = ''
    for item in json.loads(files)['items']:
        if '_metrics' in item['path']:
            print(item['path'])
            last_file_path = f'gds://{item["volumeName"]}{item["path"]}'
            os.makedirs(os.path.dirname(f'metric_files{item["path"]}'), exist_ok=True)
            metric_file = icav1.download_gds_file(last_file_path, token=os.getenv('ICA_V1_TOKEN'))
            with open(f'metric_files{item["path"]}', 'w') as f:
                f.write(metric_file)

    file_dir = f"metric_files/{last_file_path[len('gds://'):].rsplit('/', 1)[0].split('/', 1)[-1]}"
    print(file_dir)
    file_list = [f'{file_dir}/{x}' for x in os.listdir(file_dir)]
    # print(file_list)
    macam.make_contractual_and_all_metrics(sample_id=pair[1], file_path=file_dir, csv_metrics_files=file_list, output_dir=file_dir)

    for file in glob(f'{file_dir}/*_metrics.csv'):
        os.remove(file)

/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.cnv_metrics.csv
download_gds_file gds://bssh.69fbb80cfe7635369366247c9c8b1a82/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.cnv_metrics.csv...
/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.fastqc_metrics.csv
download_gds_file gds://bssh.69fbb80cfe7635369366247c9c8b1a82/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.fastqc_metrics.csv...
/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.gc_metrics.csv
download_gds_file gds://bssh.69fbb80cfe7635369366247c9c8b1a82/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.gc_metrics.csv...
/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.mapping_metrics.csv
download_gds_file gds://bssh.69fbb80cfe7635369366247c9c8b1a82/datasets/app.aps1-sh-prod.491493/dragen-germline/output/NPM1000B52_90119.mapping_metrics.csv...
/datasets/ap

In [4]:
import shutil

if os.path.exists('metric_files.zip'):
    os.remove('metric_files.zip')
shutil.make_archive('metric_files', 'zip', 'metric_files')

'/data/metric_files.zip'

In [5]:
from datetime import datetime
from scripts.external import s3
from scripts.config import METRICS_BUCKET

now = datetime.now()
prefix = now.strftime('%Y%m%d%H%M%S.%f')[:-3]

with open('metric_files.zip', 'rb') as fp:
    s3.put_to_s3(fp, prefix, 'metric_files.zip')
